# w261 Final Project - Clickthrough Rate Prediction

25   
Justin Trobec, Jeff Li, Sonya Chen, Karthik Srinivasan
Spring 2021, section 5, Team 25

## Table of Contents

* __Section 1__ - Question Formulation
* __Section 2__ - Algorithm Explanation
* __Section 3__ - EDA & Challenges
* __Section 4__ - Algorithm Implementation
* __Section 5__ - Course Concepts

In [0]:
## imports

from pyspark.sql import functions as f
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, IntegerType, NullType, ShortType, DateType, BooleanType, BinaryType
from pyspark.sql import SQLContext

sqlContext = SQLContext(sc)

In [0]:
display(dbutils.fs.ls("dbfs:/mnt/mids-w261/datasets_final_project/"))

path,name,size
dbfs:/mnt/mids-w261/datasets_final_project/airlines/,airlines/,0
dbfs:/mnt/mids-w261/datasets_final_project/airlines_data/,airlines_data/,0
dbfs:/mnt/mids-w261/datasets_final_project/allstate-claims-severity.zip,allstate-claims-severity.zip,51204863
dbfs:/mnt/mids-w261/datasets_final_project/dac.tar.gz,dac.tar.gz,4576820670
dbfs:/mnt/mids-w261/datasets_final_project/kdd-cup-2014-predicting-excitement-at-donors-choose.zip,kdd-cup-2014-predicting-excitement-at-donors-choose.zip,971133938
dbfs:/mnt/mids-w261/datasets_final_project/parquet_airlines_data/,parquet_airlines_data/,0
dbfs:/mnt/mids-w261/datasets_final_project/parquet_airlines_data_3m/,parquet_airlines_data_3m/,0
dbfs:/mnt/mids-w261/datasets_final_project/parquet_airlines_data_6m/,parquet_airlines_data_6m/,0
dbfs:/mnt/mids-w261/datasets_final_project/porto-seguro-safe-driver-prediction.zip,porto-seguro-safe-driver-prediction.zip,80247571
dbfs:/mnt/mids-w261/datasets_final_project/walmart-recruiting-trip-type-classification.zip,walmart-recruiting-trip-type-classification.zip,11510035


In [0]:
display(dbutils.fs.ls("dbfs:/mnt/mids-w261/"))

path,name,size
dbfs:/mnt/mids-w261/10/,10/,0
dbfs:/mnt/mids-w261/BLE_Project/,BLE_Project/,0
dbfs:/mnt/mids-w261/DEMO10/,DEMO10/,0
dbfs:/mnt/mids-w261/DEMO11/,DEMO11/,0
dbfs:/mnt/mids-w261/DEMO8/,DEMO8/,0
dbfs:/mnt/mids-w261/DEMO9/,DEMO9/,0
dbfs:/mnt/mids-w261/HW2/,HW2/,0
dbfs:/mnt/mids-w261/HW3/,HW3/,0
dbfs:/mnt/mids-w261/HW4/,HW4/,0
dbfs:/mnt/mids-w261/HW5/,HW5/,0


In [0]:
display(dbutils.fs.ls("dbfs:/mnt/mids-w261/datasets_final_project/weather_data"))

path,name,size
dbfs:/mnt/mids-w261/datasets_final_project/weather_data/teamjvml1/,teamjvml1/,0
dbfs:/mnt/mids-w261/datasets_final_project/weather_data/weather-miss.parquet/,weather-miss.parquet/,0
dbfs:/mnt/mids-w261/datasets_final_project/weather_data/weather2015a.parquet/,weather2015a.parquet/,0
dbfs:/mnt/mids-w261/datasets_final_project/weather_data/weather2016a.parquet/,weather2016a.parquet/,0
dbfs:/mnt/mids-w261/datasets_final_project/weather_data/weather2017a.parquet/,weather2017a.parquet/,0
dbfs:/mnt/mids-w261/datasets_final_project/weather_data/weather2018a.parquet/,weather2018a.parquet/,0
dbfs:/mnt/mids-w261/datasets_final_project/weather_data/weather2019a.parquet/,weather2019a.parquet/,0


In [0]:
display(dbutils.fs.ls("dbfs:/mnt/mids-w261/datasets_final_project/parquet_airlines_data/2015.parquet/"))

path,name,size
dbfs:/mnt/mids-w261/datasets_final_project/parquet_airlines_data/2015.parquet/_SUCCESS,_SUCCESS,0
dbfs:/mnt/mids-w261/datasets_final_project/parquet_airlines_data/2015.parquet/part-00000-58942cdd-b6e2-478a-879d-5e55b22d2e6c-c000.snappy.parquet,part-00000-58942cdd-b6e2-478a-879d-5e55b22d2e6c-c000.snappy.parquet,9951195
dbfs:/mnt/mids-w261/datasets_final_project/parquet_airlines_data/2015.parquet/part-00001-58942cdd-b6e2-478a-879d-5e55b22d2e6c-c000.snappy.parquet,part-00001-58942cdd-b6e2-478a-879d-5e55b22d2e6c-c000.snappy.parquet,10170686
dbfs:/mnt/mids-w261/datasets_final_project/parquet_airlines_data/2015.parquet/part-00002-58942cdd-b6e2-478a-879d-5e55b22d2e6c-c000.snappy.parquet,part-00002-58942cdd-b6e2-478a-879d-5e55b22d2e6c-c000.snappy.parquet,10220358
dbfs:/mnt/mids-w261/datasets_final_project/parquet_airlines_data/2015.parquet/part-00003-58942cdd-b6e2-478a-879d-5e55b22d2e6c-c000.snappy.parquet,part-00003-58942cdd-b6e2-478a-879d-5e55b22d2e6c-c000.snappy.parquet,9919462
dbfs:/mnt/mids-w261/datasets_final_project/parquet_airlines_data/2015.parquet/part-00004-58942cdd-b6e2-478a-879d-5e55b22d2e6c-c000.snappy.parquet,part-00004-58942cdd-b6e2-478a-879d-5e55b22d2e6c-c000.snappy.parquet,10135365
dbfs:/mnt/mids-w261/datasets_final_project/parquet_airlines_data/2015.parquet/part-00005-58942cdd-b6e2-478a-879d-5e55b22d2e6c-c000.snappy.parquet,part-00005-58942cdd-b6e2-478a-879d-5e55b22d2e6c-c000.snappy.parquet,10182383
dbfs:/mnt/mids-w261/datasets_final_project/parquet_airlines_data/2015.parquet/part-00006-58942cdd-b6e2-478a-879d-5e55b22d2e6c-c000.snappy.parquet,part-00006-58942cdd-b6e2-478a-879d-5e55b22d2e6c-c000.snappy.parquet,10279114
dbfs:/mnt/mids-w261/datasets_final_project/parquet_airlines_data/2015.parquet/part-00007-58942cdd-b6e2-478a-879d-5e55b22d2e6c-c000.snappy.parquet,part-00007-58942cdd-b6e2-478a-879d-5e55b22d2e6c-c000.snappy.parquet,10159971
dbfs:/mnt/mids-w261/datasets_final_project/parquet_airlines_data/2015.parquet/part-00008-58942cdd-b6e2-478a-879d-5e55b22d2e6c-c000.snappy.parquet,part-00008-58942cdd-b6e2-478a-879d-5e55b22d2e6c-c000.snappy.parquet,9979359


In [0]:
airlines = spark.read.option("header", "true").parquet(f"dbfs:/mnt/mids-w261/datasets_final_project/parquet_airlines_data/201*.parquet")
display(airlines.sample(False, 0.00001))

YEAR,QUARTER,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,FL_DATE,OP_UNIQUE_CARRIER,OP_CARRIER_AIRLINE_ID,OP_CARRIER,TAIL_NUM,OP_CARRIER_FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN_AIRPORT_SEQ_ID,ORIGIN_CITY_MARKET_ID,ORIGIN,ORIGIN_CITY_NAME,ORIGIN_STATE_ABR,ORIGIN_STATE_FIPS,ORIGIN_STATE_NM,ORIGIN_WAC,DEST_AIRPORT_ID,DEST_AIRPORT_SEQ_ID,DEST_CITY_MARKET_ID,DEST,DEST_CITY_NAME,DEST_STATE_ABR,DEST_STATE_FIPS,DEST_STATE_NM,DEST_WAC,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,DEP_DELAY_NEW,DEP_DEL15,DEP_DELAY_GROUP,DEP_TIME_BLK,TAXI_OUT,WHEELS_OFF,WHEELS_ON,TAXI_IN,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,ARR_DELAY_NEW,ARR_DEL15,ARR_DELAY_GROUP,ARR_TIME_BLK,CANCELLED,CANCELLATION_CODE,DIVERTED,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,FLIGHTS,DISTANCE,DISTANCE_GROUP,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,FIRST_DEP_TIME,TOTAL_ADD_GTIME,LONGEST_ADD_GTIME,DIV_AIRPORT_LANDINGS,DIV_REACHED_DEST,DIV_ACTUAL_ELAPSED_TIME,DIV_ARR_DELAY,DIV_DISTANCE,DIV1_AIRPORT,DIV1_AIRPORT_ID,DIV1_AIRPORT_SEQ_ID,DIV1_WHEELS_ON,DIV1_TOTAL_GTIME,DIV1_LONGEST_GTIME,DIV1_WHEELS_OFF,DIV1_TAIL_NUM,DIV2_AIRPORT,DIV2_AIRPORT_ID,DIV2_AIRPORT_SEQ_ID,DIV2_WHEELS_ON,DIV2_TOTAL_GTIME,DIV2_LONGEST_GTIME,DIV2_WHEELS_OFF,DIV2_TAIL_NUM,DIV3_AIRPORT,DIV3_AIRPORT_ID,DIV3_AIRPORT_SEQ_ID,DIV3_WHEELS_ON,DIV3_TOTAL_GTIME,DIV3_LONGEST_GTIME,DIV3_WHEELS_OFF,DIV3_TAIL_NUM,DIV4_AIRPORT,DIV4_AIRPORT_ID,DIV4_AIRPORT_SEQ_ID,DIV4_WHEELS_ON,DIV4_TOTAL_GTIME,DIV4_LONGEST_GTIME,DIV4_WHEELS_OFF,DIV4_TAIL_NUM,DIV5_AIRPORT,DIV5_AIRPORT_ID,DIV5_AIRPORT_SEQ_ID,DIV5_WHEELS_ON,DIV5_TOTAL_GTIME,DIV5_LONGEST_GTIME,DIV5_WHEELS_OFF,DIV5_TAIL_NUM
2019,2,6,21,5,2019-06-21,YV,20378,YV,N88310,6314,11140,1114008,31140,CRP,"Corpus Christi, TX",TX,48,Texas,74,12266,1226603,31453,IAH,"Houston, TX",TX,48,Texas,74,1756,1745,-11.0,0.0,0.0,-1,1700-1759,9.0,1754,1831,6.0,1856,1837,-19.0,0.0,0.0,-2,1800-1859,0.0,null,0.0,60.0,52.0,37.0,1.0,201.0,1,null,null,null,null,null,null,null,null,0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
2019,2,6,15,6,2019-06-15,WN,19393,WN,N267WN,5734,13204,1320402,31454,MCO,"Orlando, FL",FL,12,Florida,33,10423,1042302,30423,AUS,"Austin, TX",TX,48,Texas,74,840,838,-2.0,0.0,0.0,-1,0800-0859,14.0,852,1004,3.0,1015,1007,-8.0,0.0,0.0,-1,1000-1059,0.0,null,0.0,155.0,149.0,132.0,1.0,994.0,4,null,null,null,null,null,null,null,null,0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
2019,2,6,18,2,2019-06-18,WN,19393,WN,N902WN,2376,12339,1233904,32337,IND,"Indianapolis, IN",IN,18,Indiana,42,13204,1320402,31454,MCO,"Orlando, FL",FL,12,Florida,33,2005,1957,-8.0,0.0,0.0,-1,2000-2059,9.0,2006,2155,8.0,2225,2203,-22.0,0.0,0.0,-2,2200-2259,0.0,null,0.0,140.0,126.0,109.0,1.0,829.0,4,null,null,null,null,null,null,null,null,0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
2019,2,6,2,7,2019-06-02,B6,20409,B6,N534JB,2806,10693,1069302,30693,BNA,"Nashville, TN",TN,47,Tennessee,54,10721,1072102,30721,BOS,"Boston, MA",MA,25,Massachusetts,13,1500,1500,0.0,0.0,0.0,0,1500-1559,14.0,1514,1851,12.0,1834,1903,29.0,29.0,1.0,1,1800-1859,0.0,null,0.0,154.0,183.0,157.0,1.0,942.0,4,0.0,0.0,29.0,0.0,0.0,null,null,null,0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
2019,2,6,2,7,2019-06-02,WN,19393,WN,null,3942,13204,1320402,31454,MCO,"Orlando, FL",FL,12,Florida,33,12391,1239103,31703,ISP,"Islip, NY",NY,36,New York,22,1715,null,null,null,null,null,

In [0]:
airlines.printSchema()

root
-- YEAR: integer (nullable = true)
-- QUARTER: integer (nullable = true)
-- MONTH: integer (nullable = true)
-- DAY_OF_MONTH: integer (nullable = true)
-- DAY_OF_WEEK: integer (nullable = true)
-- FL_DATE: string (nullable = true)
-- OP_UNIQUE_CARRIER: string (nullable = true)
-- OP_CARRIER_AIRLINE_ID: integer (nullable = true)
-- OP_CARRIER: string (nullable = true)
-- TAIL_NUM: string (nullable = true)
-- OP_CARRIER_FL_NUM: integer (nullable = true)
-- ORIGIN_AIRPORT_ID: integer (nullable = true)
-- ORIGIN_AIRPORT_SEQ_ID: integer (nullable = true)
-- ORIGIN_CITY_MARKET_ID: integer (nullable = true)
-- ORIGIN: string (nullable = true)
-- ORIGIN_CITY_NAME: string (nullable = true)
-- ORIGIN_STATE_ABR: string (nullable = true)
-- ORIGIN_STATE_FIPS: integer (nullable = true)
-- ORIGIN_STATE_NM: string (nullable = true)
-- ORIGIN_WAC: integer (nullable = true)
-- DEST_AIRPORT_ID: integer (nullable = true)
-- DEST_AIRPORT_SEQ_ID: integer (nullable = true)
-- DEST_CITY_MARKET_ID: integer (nullable = true)
-- DEST: string (nullable = true)
-- DEST_CITY_NAME: string (nullable = true)
-- DEST_STATE_ABR: string (nullable = true)
-- DEST_STATE_FIPS: integer (nullable = true)
-- DEST_STATE_NM: string (nullable = true)
-- DEST_WAC: integer (nullable = true)
-- CRS_DEP_TIME: integer (nullable = true)
-- DEP_TIME: integer (nullable = true)
-- DEP_DELAY: double (nullable = true)
-- DEP_DELAY_NEW: double (nullable = true)
-- DEP_DEL15: double (nullable = true)
-- DEP_DELAY_GROUP: integer (nullable = true)
-- DEP_TIME_BLK: string (nullable = true)
-- TAXI_OUT: double (nullable = true)
-- WHEELS_OFF: integer (nullable = true)
-- WHEELS_ON: integer (nullable = true)
-- TAXI_IN: double (nullable = true)
-- CRS_ARR_TIME: integer (nullable = true)
-- ARR_TIME: integer (nullable = true)
-- ARR_DELAY: double (nullable = true)
-- ARR_DELAY_NEW: double (nullable = true)
-- ARR_DEL15: double (nullable = true)
-- ARR_DELAY_GROUP: integer (nullable = true)
-- ARR_TIME_BLK: string (nullable = true)
-- CANCELLED: double (nullable = true)
-- CANCELLATION_CODE: string (nullable = true)
-- DIVERTED: double (nullable = true)
-- CRS_ELAPSED_TIME: double (nullable = true)
-- ACTUAL_ELAPSED_TIME: double (nullable = true)
-- AIR_TIME: double (nullable = true)
-- FLIGHTS: double (nullable = true)
-- DISTANCE: double (nullable = true)
-- DISTANCE_GROUP: integer (nullable = true)
-- CARRIER_DELAY: double (nullable = true)
-- WEATHER_DELAY: double (nullable = true)
-- NAS_DELAY: double (nullable = true)
-- SECURITY_DELAY: double (nullable = true)
-- LATE_AIRCRAFT_DELAY: double (nullable = true)
-- FIRST_DEP_TIME: integer (nullable = true)
-- TOTAL_ADD_GTIME: double (nullable = true)
-- LONGEST_ADD_GTIME: double (nullable = true)
-- DIV_AIRPORT_LANDINGS: integer (nullable = true)
-- DIV_REACHED_DEST: double (nullable = true)
-- DIV_ACTUAL_ELAPSED_TIME: double (nullable = true)
-- DIV_ARR_DELAY: double (nullable = true)
-- DIV_DISTANCE: double (nullable = true)
-- DIV1_AIRPORT: string (nullable = true)
-- DIV1_AIRPORT_ID: integer (nullable = true)
-- DIV1_AIRPORT_SEQ_ID: integer (nullable = true)
-- DIV1_WHEELS_ON: integer (nullable = true)
-- DIV1_TOTAL_GTIME: double (nullable = true)
-- DIV1_LONGEST_GTIME: double (nullable = true)
-- DIV1_WHEELS_OFF: integer (nullable = true)
-- DIV1_TAIL_NUM: string (nullable = true)
-- DIV2_AIRPORT: string (nullable = true)
-- DIV2_AIRPORT_ID: integer (nullable = true)
-- DIV2_AIRPORT_SEQ_ID: integer (nullable = true)
-- DIV2_WHEELS_ON: integer (nullable = true)
-- DIV2_TOTAL_GTIME: double (nullable = true)
-- DIV2_LONGEST_GTIME: double (nullable = true)
-- DIV2_WHEELS_OFF: integer (nullable = true)
-- DIV2_TAIL_NUM: string (nullable = true)
-- DIV3_AIRPORT: string (nullable = true)
-- DIV3_AIRPORT_ID: integer (nullable = true)
-- DIV3_AIRPORT_SEQ_ID: integer (nullable = true)
-- DIV3_WHEELS_ON: integer (nullable = true)
-- DIV3_TOTAL_GTIME: double (nullable = true)
-- DIV3_LONGEST_GTIME: double (nullable = true)
-- DIV3_WHEELS_

In [0]:
f'number of airline record: {airlines.count():,}'

Out[18]: 'number of airline record: 31,746,841'

In [0]:
#### For the first phase of the project you will focus on flights departing from two major US airports (ORD (Chicago O'Hare) and ATL (Atlanta) for the first quarter of 2015 (that is about 160k flights)

## 161507 rows
airline_dataset = airlines.filter("(ORIGIN == 'ORD' or ORIGIN == 'ATL') AND YEAR == 2015 AND QUARTER == 1").distinct().cache()


#display(airline_dataset.sample(False, 0.00001))

In [0]:
airline_dataset.take(1)

Out[20]: [Row(YEAR=2015, QUARTER=1, MONTH=2, DAY_OF_MONTH=13, DAY_OF_WEEK=5, FL_DATE='2015-02-13', OP_UNIQUE_CARRIER='AA', OP_CARRIER_AIRLINE_ID=19805, OP_CARRIER='AA', TAIL_NUM='N3DLAA', OP_CARRIER_FL_NUM=395, ORIGIN_AIRPORT_ID=13930, ORIGIN_AIRPORT_SEQ_ID=1393003, ORIGIN_CITY_MARKET_ID=30977, ORIGIN='ORD', ORIGIN_CITY_NAME='Chicago, IL', ORIGIN_STATE_ABR='IL', ORIGIN_STATE_FIPS=17, ORIGIN_STATE_NM='Illinois', ORIGIN_WAC=41, DEST_AIRPORT_ID=14843, DEST_AIRPORT_SEQ_ID=1484304, DEST_CITY_MARKET_ID=34819, DEST='SJU', DEST_CITY_NAME='San Juan, PR', DEST_STATE_ABR='PR', DEST_STATE_FIPS=72, DEST_STATE_NM='Puerto Rico', DEST_WAC=3, CRS_DEP_TIME=830, DEP_TIME=823, DEP_DELAY=-7.0, DEP_DELAY_NEW=0.0, DEP_DEL15=0.0, DEP_DELAY_GROUP=-1, DEP_TIME_BLK='0800-0859', TAXI_OUT=16.0, WHEELS_OFF=839, WHEELS_ON=1436, TAXI_IN=8.0, CRS_ARR_TIME=1502, ARR_TIME=1444, ARR_DELAY=-18.0, ARR_DELAY_NEW=0.0, ARR_DEL15=0.0, ARR_DELAY_GROUP=-2, ARR_TIME_BLK='1500-1559', CANCELLED=0.0, CANCELLATION_CODE=None, DIVERTED=0.0, CRS_ELAPSED_TIME=272.0, ACTUAL_ELAPSED_TIME=261.0, AIR_TIME=237.0, FLIGHTS=1.0, DISTANCE=2072.0, DISTANCE_GROUP=9, CARRIER_DELAY=None, WEATHER_DELAY=None, NAS_DELAY=None, SECURITY_DELAY=None, LATE_AIRCRAFT_DELAY=None, FIRST_DEP_TIME=None, TOTAL_ADD_GTIME=None, LONGEST_ADD_GTIME=None, DIV_AIRPORT_LANDINGS=0, DIV_REACHED_DEST=None, DIV_ACTUAL_ELAPSED_TIME=None, DIV_ARR_DELAY=None, DIV_DISTANCE=None, DIV1_AIRPORT=None, DIV1_AIRPORT_ID=None, DIV1_AIRPORT_SEQ_ID=None, DIV1_WHEELS_ON=None, DIV1_TOTAL_GTIME=None, DIV1_LONGEST_GTIME=None, DIV1_WHEELS_OFF=None, DIV1_TAIL_NUM=None, DIV2_AIRPORT=None, DIV2_AIRPORT_ID=None, DIV2_AIRPORT_SEQ_ID=None, DIV2_WHEELS_ON=None, DIV2_TOTAL_GTIME=None, DIV2_LONGEST_GTIME=None, DIV2_WHEELS_OFF=None, DIV2_TAIL_NUM=None, DIV3_AIRPORT=None, DIV3_AIRPORT_ID=None, DIV3_AIRPORT_SEQ_ID=None, DIV3_WHEELS_ON=None, DIV3_TOTAL_GTIME=None, DIV3_LONGEST_GTIME=None, DIV3_WHEELS_OFF=None, DIV3_TAIL_NUM=None, DIV4_AIRPORT=None, DIV4_AIRPORT_ID=None, DIV4_AIRPORT_SEQ_ID=None, DIV4_WHEELS_ON=None, DIV4_TOTAL_GTIME=None, DIV4_LONGEST_GTIME=None, DIV4_WHEELS_OFF=None, DIV4_TAIL_NUM=None, DIV5_AIRPORT=None, DIV5_AIRPORT_ID=None, DIV5_AIRPORT_SEQ_ID=None, DIV5_WHEELS_ON=None, DIV5_TOTAL_GTIME=None, DIV5_LONGEST_GTIME=None, DIV5_WHEELS_OFF=None, DIV5_TAIL_NUM=None)]

In [0]:
display(airline_dataset.describe())

summary,YEAR,QUARTER,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,FL_DATE,OP_UNIQUE_CARRIER,OP_CARRIER_AIRLINE_ID,OP_CARRIER,TAIL_NUM,OP_CARRIER_FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN_AIRPORT_SEQ_ID,ORIGIN_CITY_MARKET_ID,ORIGIN,ORIGIN_CITY_NAME,ORIGIN_STATE_ABR,ORIGIN_STATE_FIPS,ORIGIN_STATE_NM,ORIGIN_WAC,DEST_AIRPORT_ID,DEST_AIRPORT_SEQ_ID,DEST_CITY_MARKET_ID,DEST,DEST_CITY_NAME,DEST_STATE_ABR,DEST_STATE_FIPS,DEST_STATE_NM,DEST_WAC,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,DEP_DELAY_NEW,DEP_DEL15,DEP_DELAY_GROUP,DEP_TIME_BLK,TAXI_OUT,WHEELS_OFF,WHEELS_ON,TAXI_IN,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,ARR_DELAY_NEW,ARR_DEL15,ARR_DELAY_GROUP,ARR_TIME_BLK,CANCELLED,CANCELLATION_CODE,DIVERTED,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,FLIGHTS,DISTANCE,DISTANCE_GROUP,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,FIRST_DEP_TIME,TOTAL_ADD_GTIME,LONGEST_ADD_GTIME,DIV_AIRPORT_LANDINGS,DIV_REACHED_DEST,DIV_ACTUAL_ELAPSED_TIME,DIV_ARR_DELAY,DIV_DISTANCE,DIV1_AIRPORT,DIV1_AIRPORT_ID,DIV1_AIRPORT_SEQ_ID,DIV1_WHEELS_ON,DIV1_TOTAL_GTIME,DIV1_LONGEST_GTIME,DIV1_WHEELS_OFF,DIV1_TAIL_NUM,DIV2_AIRPORT,DIV2_AIRPORT_ID,DIV2_AIRPORT_SEQ_ID,DIV2_WHEELS_ON,DIV2_TOTAL_GTIME,DIV2_LONGEST_GTIME,DIV2_WHEELS_OFF,DIV2_TAIL_NUM,DIV3_AIRPORT,DIV3_AIRPORT_ID,DIV3_AIRPORT_SEQ_ID,DIV3_WHEELS_ON,DIV3_TOTAL_GTIME,DIV3_LONGEST_GTIME,DIV3_WHEELS_OFF,DIV3_TAIL_NUM,DIV4_AIRPORT,DIV4_AIRPORT_ID,DIV4_AIRPORT_SEQ_ID,DIV4_WHEELS_ON,DIV4_TOTAL_GTIME,DIV4_LONGEST_GTIME,DIV4_WHEELS_OFF,DIV4_TAIL_NUM,DIV5_AIRPORT,DIV5_AIRPORT_ID,DIV5_AIRPORT_SEQ_ID,DIV5_WHEELS_ON,DIV5_TOTAL_GTIME,DIV5_LONGEST_GTIME,DIV5_WHEELS_OFF,DIV5_TAIL_NUM
count,161057,161057,161057,161057,161057,161057,161057,161057,161057,160375,161057,161057,161057,161057,161057,161057,161057,161057,161057,161057,161057,161057,161057,161057,161057,161057,161057,161057,161057,161057,156231,156231,156231,156231,156231,161057,156121,156121,155957,155957,161057,155957,155692,155692,155692,155692,161057,161057,4984,161057,161057,155692,155692,161057,161057,161057,34915,34915,34915,34915,34915,1053,1053,1053,161057,381,265,265,381,429,429,429,429,429,429,278,278,17,17,17,17,17,17,5,5,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
mean,2015.0,1.0,2.0365522765232185,15.591504870946311,3.9165202381765463,null,null,20010.91815320042,null,null,2508.798711015355,11963.80259162905,1196384.3722098388,30654.21638922866,null,null,null,14.773906132611435,null,37.10433573207001,12707.705110613013,1270773.0347454627,32018.14314807801,null,null,null,28.618718838672024,null,49.025475452790005,1422.252767653688,1437.2168967746477,12.583827793459685,14.368870454647285,0.2348125532064699,0.2524274951834143,null,18.062086458580204,1465.591502744666,1563.5781721884878,6.14908596600345,1584.805360834984,1566.3260642356547,7.511567710608124,14.163483030598874,0.22425686612028878,0.00872877219124939,null,0.030945565855566664,null,0.0023656221089427966,124.43478395847433,120.03714384811038,95.8361765537086,1.0,662.2313404571053,3.1245459681975944,19.903050264929114,6.769039094944866,12.16838035228412,0.017556923958184163,19.076671917513963,1399.9924026590693,36.22602089268756,35.970560303893635,0.005116201096506206,0.6955380577427821,347.10188679245283,232.63396226415094,47.90813648293963,null,12499.242424242424,1249927.13986014,1464.04662004662,27.405594405594407,22.242424242424242,1474.8848920863309,null,null,12674.176470588236,1267420.8823529412,1341.4117647058824,11.941176470588236,10.235294117647058,1392.2,null,null,10397.0,1039705.0,51.0,4.0,4.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
stddev,0.0,0.0,0.8326601487890225,8.679527916539659,1.9958147997611475,null,null,314.62649685727985,null,null,1746.4192642359585,1755.181564074375,175517.16281470386,288.1418927719043,null,null,null,1.987185467392442,null,3.477574567936774,1522.4292803723658,152242.81778366142,1388.3480346900362,null,null,null,16.113962414973802,null,21.488663740357904,453.9704741890777,463.8574610933501,36.393692984765096,35.5969485

# Weather
https://data.nodc.noaa.gov/cgi-bin/iso?id=gov.noaa.ncdc:C00532

In [0]:
display(dbutils.fs.ls("dbfs:/mnt/mids-w261/datasets_final_project/weather_data"))

path,name,size
dbfs:/mnt/mids-w261/datasets_final_project/weather_data/teamjvml1/,teamjvml1/,0
dbfs:/mnt/mids-w261/datasets_final_project/weather_data/weather-miss.parquet/,weather-miss.parquet/,0
dbfs:/mnt/mids-w261/datasets_final_project/weather_data/weather2015a.parquet/,weather2015a.parquet/,0
dbfs:/mnt/mids-w261/datasets_final_project/weather_data/weather2016a.parquet/,weather2016a.parquet/,0
dbfs:/mnt/mids-w261/datasets_final_project/weather_data/weather2017a.parquet/,weather2017a.parquet/,0
dbfs:/mnt/mids-w261/datasets_final_project/weather_data/weather2018a.parquet/,weather2018a.parquet/,0
dbfs:/mnt/mids-w261/datasets_final_project/weather_data/weather2019a.parquet/,weather2019a.parquet/,0


In [0]:
weather = spark.read.option("header", "true")\
                    .parquet(f"dbfs:/mnt/mids-w261/datasets_final_project/weather_data/*.parquet")

f'{weather.count():,}'

Out[23]: '630,904,436'

In [0]:
weather.select(f.col('CALL_SIGN')).distinct().collect()

Out[24]: [Row(CALL_SIGN='KTZR '),
 Row(CALL_SIGN='KMRN '),
 Row(CALL_SIGN='PALJ '),
 Row(CALL_SIGN='PADE '),
 Row(CALL_SIGN='PAOR '),
 Row(CALL_SIGN='K1A6 '),
 Row(CALL_SIGN='PAAK '),
 Row(CALL_SIGN='K2G4 '),
 Row(CALL_SIGN='PATO '),
 Row(CALL_SIGN='KW99 '),
 Row(CALL_SIGN='PASD '),
 Row(CALL_SIGN='PABV '),
 Row(CALL_SIGN='KINJ '),
 Row(CALL_SIGN='PAIG '),
 Row(CALL_SIGN='PAOT '),
 Row(CALL_SIGN='KS32 '),
 Row(CALL_SIGN='KPNA '),
 Row(CALL_SIGN='KMZG '),
 Row(CALL_SIGN='KNBT '),
 Row(CALL_SIGN='KHSB '),
 Row(CALL_SIGN='KARM '),
 Row(CALL_SIGN='PAPO '),
 Row(CALL_SIGN='PASN '),
 Row(CALL_SIGN='KT82 '),
 Row(CALL_SIGN='PABA '),
 Row(CALL_SIGN='KCBE '),
 Row(CALL_SIGN='PAJC '),
 Row(CALL_SIGN='KSAR '),
 Row(CALL_SIGN='KEMM '),
 Row(CALL_SIGN='KLVJ '),
 Row(CALL_SIGN='PAII '),
 Row(CALL_SIGN='PAIK '),
 Row(CALL_SIGN='SFD '),
 Row(CALL_SIGN='KY51 '),
 Row(CALL_SIGN='K40J '),
 Row(CALL_SIGN='PAAQ '),
 Row(CALL_SIGN='PAWD '),
 Row(CALL_SIGN='K2D5 '),
 Row(CALL_SIGN='KJFZ '),
 Row(CALL_SIGN='KRBO '),
 Row(CALL_SIGN='KSBD '),
 Row(CALL_SIGN='KLUD '),
 Row(CALL_SIGN='PAEI '),
 Row(CALL_SIGN='PAWI '),
 Row(CALL_SIGN='KBPC '),
 Row(CALL_SIGN='PASA '),
 Row(CALL_SIGN='KPSF '),
 Row(CALL_SIGN='PAEH '),
 Row(CALL_SIGN='KMHT '),
 Row(CALL_SIGN='KFTK '),
 Row(CALL_SIGN='KDVN '),
 Row(CALL_SIGN='KPCW '),
 Row(CALL_SIGN='K2I0 '),
 Row(CALL_SIGN='K1J0 '),
 Row(CALL_SIGN='KIIY '),
 Row(CALL_SIGN='KCLK '),
 Row(CALL_SIGN='KJAS '),
 Row(CALL_SIGN='KRNP '),
 Row(CALL_SIGN='PAKV '),
 Row(CALL_SIGN='PAFR '),
 Row(CALL_SIGN='KCFD '),
 Row(CALL_SIGN='KNXP '),
 Row(CALL_SIGN='KAPY '),
 Row(CALL_SIGN='KVTI '),
 Row(CALL_SIGN='K7R3 '),
 Row(CALL_SIGN='KMLJ '),
 Row(CALL_SIGN='PAHP '),
 Row(CALL_SIGN='PAIN '),
 Row(CALL_SIGN='KOOA '),
 Row(CALL_SIGN='K11R '),
 Row(CALL_SIGN='PARY '),
 Row(CALL_SIGN='KARR '),
 Row(CALL_SIGN='PALG '),
 Row(CALL_SIGN='KI35 '),
 Row(CALL_SIGN='KVSH '),
 Row(CALL_SIGN='KTKC '),
 Row(CALL_SIGN='PAFE '),
 Row(CALL_SIGN='KGEV '),
 Row(CALL_SIGN='KAEJ '),
 Row(CALL_SIGN='PAPB '),
 Row(CALL_SIGN='KRCE '),
 Row(CALL_SIGN='PALU '),
 Row(CALL_SIGN='KFWB '),
 Row(CALL_SIGN='PAVL '),
 Row(CALL_SIGN='KLCH '),
 Row(CALL_SIGN='KMCJ '),
 Row(CALL_SIGN='PANW '),
 Row(CALL_SIGN='PFSH '),
 Row(CALL_SIGN='PASH '),
 Row(CALL_SIGN='KCLT '),
 Row(CALL_SIGN='PASI '),
 Row(CALL_SIGN='PAEM '),
 Row(CALL_SIGN='PABR '),
 Row(CALL_SIGN='K6A1 '),
 Row(CALL_SIGN='KJWY '),
 Row(CALL_SIGN='PAGS '),
 Row(CALL_SIGN='KHUM '),
 Row(CALL_SIGN='KPVB '),
 Row(CALL_SIGN='PAKU '),
 Row(CALL_SIGN='KBVE '),
 Row(CALL_SIGN='KMZZ '),
 Row(CALL_SIGN='KOMN '),
 Row(CALL_SIGN='KRFI '),
 Row(CALL_SIGN='KNGW '),
 Row(CALL_SIGN='K3R7 '),
 Row(CALL_SIGN='PAHY '),
 Row(CALL_SIGN='KRSV '),
 Row(CALL_SIGN='K4A9 '),
 Row(CALL_SIGN='K6R3 '),
 Row(CALL_SIGN='PASK '),
 Row(CALL_SIGN='K20V '),
 Row(CALL_SIGN='KD55 '),
 Row(CALL_SIGN='K2V5 '),
 Row(CALL_SIGN='PAPH '),
 Row(CALL_SIGN='KW43 '),
 Row(CALL_SIGN='KPKB '),
 Row(CALL_SIGN='KIYA '),
 Row(CALL_SIGN='PAUN '),
 Row(CALL_SIGN='PAWN '),
 Row(CALL_SIGN='KSLH '),
 Row(CALL_SIGN='KGZL '),
 Row(CALL_SIGN='KBPT '),
 Row(CALL_SIGN='KFBR '),
 Row(CALL_SIGN='PACV '),
 Row(CALL_SIGN='KIKW '),
 Row(CALL_SIGN='KLWA '),
 Row(CALL_SIGN='KBAZ '),
 Row(CALL_SIGN='K3J7 '),
 Row(CALL_SIGN='KANQ '),
 Row(CALL_SIGN='PAQT '),
 Row(CALL_SIGN='KSAA '),
 Row(CALL_SIGN='KCWF '),
 Row(CALL_SIGN='KSZT '),
 Row(CALL_SIGN='KNMM '),
 Row(CALL_SIGN='KHTS '),
 Row(CALL_SIGN='KOXI '),
 Row(CALL_SIGN='KDYA '),
 Row(CALL_SIGN='PAGB '),
 Row(CALL_SIGN='KUKT '),
 Row(CALL_SIGN='KEZS '),
 Row(CALL_SIGN='PAIL '),
 Row(CALL_SIGN='KMDS '),
 Row(CALL_SIGN='KAKH '),
 Row(CALL_SIGN='PAGN '),
 Row(CALL_SIGN='KPIB '),
 Row(CALL_SIGN='PASC '),
 Row(CALL_SIGN='KAVL '),
 Row(CALL_SIGN='KY50 '),
 Row(CALL_SIGN='KY63 '),
 Row(CALL_SIGN='PAMY '),
 Row(CALL_SIGN='PAVD '),
 Row(CALL_SIGN='KVKS '),
 Row(CALL_SIGN='PARS '),
 Row(CALL_SIGN='STO '),
 Row(CALL_SIGN='KBKN '),
 Row(CALL_SIGN='K1A5 '),
 Row(CALL_SIGN='KGTB '),
 Row(CALL_SIGN='KLOM '),
 Row(CALL_SIGN='KLHB '),
 Row(CALL_SIGN='KD

In [0]:
display(weather.where('DATE =="DATE"'))

STATION,DATE,SOURCE,LATITUDE,LONGITUDE,ELEVATION,NAME,REPORT_TYPE,CALL_SIGN,QUALITY_CONTROL,WND,CIG,VIS,TMP,DEW,SLP,AW1,GA1,GA2,GA3,GA4,GE1,GF1,KA1,KA2,MA1,MD1,MW1,MW2,OC1,OD1,OD2,REM,EQD,AW2,AX4,GD1,AW5,GN1,AJ1,AW3,MK1,KA4,GG3,AN1,RH1,AU5,HL1,OB1,AT8,AW7,AZ1,CH1,RH3,GK1,IB1,AX1,CT1,AK1,CN2,OE1,MW5,AO1,KA3,AA3,CR1,CF2,KB2,GM1,AT5,AY2,MW6,MG1,AH6,AU2,GD2,AW4,MF1,AA1,AH2,AH3,OE3,AT6,AL2,AL3,AX5,IB2,AI3,CV3,WA1,GH1,KF1,CU2,CT3,SA1,AU1,KD2,AI5,GO1,GD3,CG3,AI1,AL1,AW6,MW4,AX6,CV1,ME1,KC2,CN1,UA1,GD5,UG2,AT3,AT4,GJ1,MV1,GA5,CT2,CG2,ED1,AE1,CO1,KE1,KB1,AI4,MW3,KG2,AA2,AX2,AY1,RH2,OE2,CU3,MH1,AM1,AU4,GA6,KG1,AU3,AT7,KD1,GL1,IA1,GG2,OD3,UG1,CB1,AI6,CI1,CV2,AZ2,AD1,AH1,WD1,AA4,KC1,IA2,CF3,AI2,AT1,GD4,AX3,AH4,KB3,CU1,CN4,AT2,CG1,CF1,GG1,MV2,CW1,GG4,AB1,AH5,CN3


In [0]:
weather.printSchema()

root
-- STATION: string (nullable = true)
-- DATE: timestamp (nullable = true)
-- SOURCE: short (nullable = true)
-- LATITUDE: double (nullable = true)
-- LONGITUDE: double (nullable = true)
-- ELEVATION: double (nullable = true)
-- NAME: string (nullable = true)
-- REPORT_TYPE: string (nullable = true)
-- CALL_SIGN: string (nullable = true)
-- QUALITY_CONTROL: string (nullable = true)
-- WND: string (nullable = true)
-- CIG: string (nullable = true)
-- VIS: string (nullable = true)
-- TMP: string (nullable = true)
-- DEW: string (nullable = true)
-- SLP: string (nullable = true)
-- AW1: string (nullable = true)
-- GA1: string (nullable = true)
-- GA2: string (nullable = true)
-- GA3: string (nullable = true)
-- GA4: string (nullable = true)
-- GE1: string (nullable = true)
-- GF1: string (nullable = true)
-- KA1: string (nullable = true)
-- KA2: string (nullable = true)
-- MA1: string (nullable = true)
-- MD1: string (nullable = true)
-- MW1: string (nullable = true)
-- MW2: string (nullable = true)
-- OC1: string (nullable = true)
-- OD1: string (nullable = true)
-- OD2: string (nullable = true)
-- REM: string (nullable = true)
-- EQD: string (nullable = true)
-- AW2: string (nullable = true)
-- AX4: string (nullable = true)
-- GD1: string (nullable = true)
-- AW5: string (nullable = true)
-- GN1: string (nullable = true)
-- AJ1: string (nullable = true)
-- AW3: string (nullable = true)
-- MK1: string (nullable = true)
-- KA4: string (nullable = true)
-- GG3: string (nullable = true)
-- AN1: string (nullable = true)
-- RH1: string (nullable = true)
-- AU5: string (nullable = true)
-- HL1: string (nullable = true)
-- OB1: string (nullable = true)
-- AT8: string (nullable = true)
-- AW7: string (nullable = true)
-- AZ1: string (nullable = true)
-- CH1: string (nullable = true)
-- RH3: string (nullable = true)
-- GK1: string (nullable = true)
-- IB1: string (nullable = true)
-- AX1: string (nullable = true)
-- CT1: string (nullable = true)
-- AK1: string (nullable = true)
-- CN2: string (nullable = true)
-- OE1: string (nullable = true)
-- MW5: string (nullable = true)
-- AO1: string (nullable = true)
-- KA3: string (nullable = true)
-- AA3: string (nullable = true)
-- CR1: string (nullable = true)
-- CF2: string (nullable = true)
-- KB2: string (nullable = true)
-- GM1: string (nullable = true)
-- AT5: string (nullable = true)
-- AY2: string (nullable = true)
-- MW6: string (nullable = true)
-- MG1: string (nullable = true)
-- AH6: string (nullable = true)
-- AU2: string (nullable = true)
-- GD2: string (nullable = true)
-- AW4: string (nullable = true)
-- MF1: string (nullable = true)
-- AA1: string (nullable = true)
-- AH2: string (nullable = true)
-- AH3: string (nullable = true)
-- OE3: string (nullable = true)
-- AT6: string (nullable = true)
-- AL2: string (nullable = true)
-- AL3: string (nullable = true)
-- AX5: string (nullable = true)
-- IB2: string (nullable = true)
-- AI3: string (nullable = true)
-- CV3: string (nullable = true)
-- WA1: string (nullable = true)
-- GH1: string (nullable = true)
-- KF1: string (nullable = true)
-- CU2: string (nullable = true)
-- CT3: string (nullable = true)
-- SA1: string (nullable = true)
-- AU1: string (nullable = true)
-- KD2: string (nullable = true)
-- AI5: string (nullable = true)
-- GO1: string (nullable = true)
-- GD3: string (nullable = true)
-- CG3: string (nullable = true)
-- AI1: string (nullable = true)
-- AL1: string (nullable = true)
-- AW6: string (nullable = true)
-- MW4: string (nullable = true)
-- AX6: string (nullable = true)
-- CV1: string (nullable = true)
-- ME1: string (nullable = true)
-- KC2: string (nullable = true)
-- CN1: string (nullable = true)
-- UA1: string (nullable = true)
-- GD5: string (nullable = true)
-- UG2: string (nullable = true)
-- AT3: string (nullable = true)
-- AT4: string (nullable = true)
-- GJ1: string (nullable = true)
-- MV1: string (nullable = true)
-- GA5: string (nullable = true)
-- CT2: string (nullable = true)
-- CG2: string

In [0]:
display(weather.sample(False, 0.00001))

STATION,DATE,SOURCE,LATITUDE,LONGITUDE,ELEVATION,NAME,REPORT_TYPE,CALL_SIGN,QUALITY_CONTROL,WND,CIG,VIS,TMP,DEW,SLP,AW1,GA1,GA2,GA3,GA4,GE1,GF1,KA1,KA2,MA1,MD1,MW1,MW2,OC1,OD1,OD2,REM,EQD,AW2,AX4,GD1,AW5,GN1,AJ1,AW3,MK1,KA4,GG3,AN1,RH1,AU5,HL1,OB1,AT8,AW7,AZ1,CH1,RH3,GK1,IB1,AX1,CT1,AK1,CN2,OE1,MW5,AO1,KA3,AA3,CR1,CF2,KB2,GM1,AT5,AY2,MW6,MG1,AH6,AU2,GD2,AW4,MF1,AA1,AH2,AH3,OE3,AT6,AL2,AL3,AX5,IB2,AI3,CV3,WA1,GH1,KF1,CU2,CT3,SA1,AU1,KD2,AI5,GO1,GD3,CG3,AI1,AL1,AW6,MW4,AX6,CV1,ME1,KC2,CN1,UA1,GD5,UG2,AT3,AT4,GJ1,MV1,GA5,CT2,CG2,ED1,AE1,CO1,KE1,KB1,AI4,MW3,KG2,AA2,AX2,AY1,RH2,OE2,CU3,MH1,AM1,AU4,GA6,KG1,AU3,AT7,KD1,GL1,IA1,GG2,OD3,UG1,CB1,AI6,CI1,CV2,AZ2,AD1,AH1,WD1,AA4,KC1,IA2,CF3,AI2,AT1,GD4,AX3,AH4,KB3,CU1,CN4,AT2,CG1,CF1,GG1,MV2,CW1,GG4,AB1,AH5,CN3
7675099999,2016-08-22T19:00:00.000+0000,4,43.384672,6.387139,80.77,"LE CANNET, FR",FM-15,99999,V020,"080,1,N,0015,1","99999,9,9,N","009999,1,9,9","+0250,1","+0100,1","99999,9",,,,,,,,,,"10220,1,99999,9",,,,,,,MET056METAR LFMC 221900Z AUTO 08003KT 9999 ////// 25/10 Q1022=,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
7675099999,2016-08-27T02:00:00.000+0000,4,43.384672,6.387139,80.77,"LE CANNET, FR",FM-12,99999,V020,"290,1,N,0005,1","99999,9,9,N","045000,1,9,9","+0181,1","+0128,1","10192,1",,,,,,,,,,"99999,9,10095,1","3,1,001,1,+999,9","00,1",,,"3,99,0010,1,999",,SYN07607675 24/83 /2901 10181 20128 30095 40192 53001 700// 333 60005 90710 91102=,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,"01,0000,9,1",,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
70305226652,2016-11-13T08:56:00.000+0000,7,61.53639,-160.34139,16.8,"KALSKAG AIRPORT, AK US",FM-15,PALG,V020,"200,5,N,0031,5","22000,5,9,N","016093,5,N,5","-0011,5","-0044,5","09872,5",,"02,5,+03658,5,99,9",,,,"9,AGL ,+99999,+99999","02,99,5,99,9,99,9,03658,1,99,9,99,9","240,M,+0011,1","240,N,-0033,1","09868,5,09848,5","1,9,005,9,+999,9",,,,,,MET12411/12/16 23:56:02 METAR PALG 130856Z 20006KT 10SM FEW120 M01/M04 A2914 RMK AO2 SLP872 60000 T10111044 400111033 51005 FZRANO,Q01 00002PRCP03Q02 00008PRCP03D01 0ADE726,,,"1,99,1,+03658,5,9",,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,"03,0000,2,1",,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
99999994085,2016-04-11T00:15:00.000+0000,null,44.0194,-100.353,647.4,"PIERRE 24 S, SD US",CRN05,99999,V020,"999,9,9,9999,9","99999,9,9,N","999999,9,9,9","+0101,1","+9999,9","99999,9",,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,"05,+0100,1,0,0371,1,0",,,,,"+0101,1,0",,,,,"05,0000,9,1",,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,"+0101,1,0",,,,,,,"+03537,1,0",,,,,,,,,,,,,,,,,,"+0101,1,0","+03570,1,0",,,"99,-06",,,,,,,,,,,,,,,,,,,,,,,,,"05,+00000,1,0",,,,,,,,,,,,,,,,,,,,,"+03610,1,0",,,,"11640,1,0,29930,1,0",,,,
99999994085,2016-11-22T17:40:00.000+0000,null,44.0194,-100.353,647.4,"PIERRE 24 S, SD US",CRN05,99999,V020,"999,9,9,9999,9","99999,9,9,N","999999,9,9,9","+0043,1","+9999,9","99999,9",,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,"05,+0042,1,0,0802,1,0",,,,,"+0043,1,0",,,,,"05,0000,9,1",,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,"+0043,1,0",,,,,,,"+02276,1,0",,,,,,,,,,,,,,,,,,"+0043,1,0","+02278,1,0",,,"99,-06",,,,,,,,,,,,,,,,,,,,,,,,,"05,+00000,1,0",,,,,,,,,,,,,,,,,,,,,"+02316,1,0",,,,"12480,1,0,29870,1,0",,,,
99999994085,2016-11-23T14:20:00.000+0000,null,44.0194,-100.353,647.4,"PIERRE 24 S, SD US",CRN05,99999,V020,"999,9,9,9999,9","99999,9,9,N","999999,9,9,9","+0002,1","+9999,9","99999,9",,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,"05,+0001,1,0,0940,1,0",,,,,"+0002,1,0",,,,,"05,0000,9,1",,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,"+0003,1,0",,,,,,,"+02275,1,0",,,,,,,,,,,,,,,,,,"+0002,1,0","+02277,1,0",,,"99,-06",,,,,,,,,,,,,,,,,,,,,,,,,"05,+00000,1,0",,,,,,,,,,,,,,,,,,,,,"+02310,1,0",,,,"00168,1,0,28760,1,0",,,,
99999994088,2016-08-31T05:15:00.000+0000,null,44.5169,-104.4363,1765.4,"SUNDANCE 8 NNW, WY US",CRN05,99999,V020,"999,9,9,9999,9","99999,9,9,N","999999,9,9,9","+01

## SQL QUERIES FOR DATABASE

In [0]:
%sql

DROP VIEW IF EXISTS WEATHER;

CREATE TEMPORARY VIEW WEATHER 
USING parquet OPTIONS (path "dbfs:/mnt/mids-w261/datasets_final_project/weather_data/weather*.parquet");

SELECT * FROM WEATHER LIMIT 10;

STATION,DATE,SOURCE,LATITUDE,LONGITUDE,ELEVATION,NAME,REPORT_TYPE,CALL_SIGN,QUALITY_CONTROL,WND,CIG,VIS,TMP,DEW,SLP,AW1,GA1,GA2,GA3,GA4,GE1,GF1,KA1,KA2,MA1,MD1,MW1,MW2,OC1,OD1,OD2,REM,EQD,AW2,AX4,GD1,AW5,GN1,AJ1,AW3,MK1,KA4,GG3,AN1,RH1,AU5,HL1,OB1,AT8,AW7,AZ1,CH1,RH3,GK1,IB1,AX1,CT1,AK1,CN2,OE1,MW5,AO1,KA3,AA3,CR1,CF2,KB2,GM1,AT5,AY2,MW6,MG1,AH6,AU2,GD2,AW4,MF1,AA1,AH2,AH3,OE3,AT6,AL2,AL3,AX5,IB2,AI3,CV3,WA1,GH1,KF1,CU2,CT3,SA1,AU1,KD2,AI5,GO1,GD3,CG3,AI1,AL1,AW6,MW4,AX6,CV1,ME1,KC2,CN1,UA1,GD5,UG2,AT3,AT4,GJ1,MV1,GA5,CT2,CG2,ED1,AE1,CO1,KE1,KB1,AI4,MW3,KG2,AA2,AX2,AY1,RH2,OE2,CU3,MH1,AM1,AU4,GA6,KG1,AU3,AT7,KD1,GL1,IA1,GG2,OD3,UG1,CB1,AI6,CI1,CV2,AZ2,AD1,AH1,WD1,AA4,KC1,IA2,CF3,AI2,AT1,GD4,AX3,AH4,KB3,CU1,CN4,AT2,CG1,CF1,GG1,MV2,CW1,GG4,AB1,AH5,CN3
7650099999,2016-01-01T00:00:00.000+0000,4,43.435555,5.213611,22.55,"PROVENCE, FR",FM-12,99999,V020,"190,1,N,0015,1","99999,9,9,N","007000,1,9,9","+0101,1","+0090,1","10255,1",,"99,9,+02250,1,99,9",,,,"9,AGL ,+99999,+99999","08,99,1,99,9,99,9,02250,1,99,9,99,9",,,"99999,9,10216,1","8,1,004,1,+999,9","61,1",,,"3,99,0026,1,999",,SYN09807650 04857 81903 10101 20090 30216 40255 58004 69901 761// 333 4/000 69907 90710 91105 555 69905=,,,,,,,"9999,1,9,999999,9,9",,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,"03,0000,2,1",,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,"06,0000,2,1",,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
7650099999,2016-01-01T00:00:00.000+0000,4,43.435555,5.213611,22.55,"PROVENCE, FR",FM-15,99999,V020,"190,1,N,0015,1","22000,1,9,N","009000,1,9,9","+0100,1","+0090,1","99999,9","61,1",,,,,,"00,99,1,99,9,99,9,99999,9,99,9,99,9",,,"10250,1,99999,9",,,,,,,MET057METAR LFML 010000Z AUTO 19003KT 9000 -RA NSC 10/09 Q1025=,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
7650099999,2016-01-01T00:30:00.000+0000,4,43.435555,5.213611,22.55,"PROVENCE, FR",FM-15,99999,V020,"250,1,N,0010,1","99999,9,9,N","008000,1,9,9","+9999,9","+9999,9","99999,9",,,,,,,,,,"10250,1,99999,9",,,,,,,MET056METAR LFML 010030Z AUTO 25002KT 8000 ///TCU 10/09 Q1025=,Q019 2ATOD,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
7650099999,2016-01-01T01:00:00.000+0000,4,43.435555,5.213611,22.55,"PROVENCE, FR",FM-12,99999,V020,"999,9,C,0000,1","99999,9,9,N","004900,1,9,9","+0099,1","+0094,1","10251,1",,"99,9,+02250,1,99,9",,,,"9,AGL ,+99999,+99999","08,99,1,99,9,99,9,02250,1,99,9,99,9",,,"99999,9,10212,1","8,1,006,1,+999,9","10,1",,,"3,99,0015,1,999",,SYN07607650 24849 80000 10099 20094 30212 40251 58006 710// 333 69925 90710 91103=,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,"01,0002,3,1",,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
7650099999,2016-01-01T01:00:00.000+0000,4,43.435555,5.213611,22.55,"PROVENCE, FR",FM-15,99999,V020,"999,9,C,0000,1","22000,1,9,N","007000,1,9,9","+0100,1","+0090,1","99999,9",,,,,,,"00,99,1,99,9,99,9,99999,9,99,9,99,9",,,"10250,1,99999,9",,,,,,,MET053METAR LFML 010100Z AUTO 00000KT 7000 NSC 10/09 Q1025=,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
7650099999,2016-01-01T01:30:00.000+0000,4,43.435555,5.213611,22.55,"PROVENCE, FR",FM-15,99999,V020,"999,9,C,0000,1","22000,1,9,N","009000,1,9,9","+0100,1","+0090,1","99999,9",,,,,,,"00,99,1,99,9,99,9,99999,9,99,9,99,9",,,"10250,1,99999,9",,,,,,,MET053METAR LFML 010130Z AUTO 00000KT 9000 NSC 10/09 Q1025=,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
7650099999,2016-01-01T02:00:00.000+0000,4,43.435555,5.213611,22.55,"PROVENCE, FR",FM-12,99999,V020,"999,9,C,0000,1","99999,9,9,N","011000,1,9,9","+0100,1","+0088,1","10254,1",,"99,9,+01750,1,99,9",,,,"9,AGL ,+99999,+99999","08,99,1,99,9,99,9,01750,1,99,9,99,9",,,"99

In [0]:
%sql

DROP VIEW IF EXISTS AIRLINES_2015;

CREATE TEMPORARY VIEW AIRLINES_2015
USING PARQUET OPTIONS (path "dbfs:/mnt/mids-w261/datasets_final_project/parquet_airlines_data/2015.parquet");

SELECT * FROM AIRLINES_2015 LIMIT 10;

-- spark.sql("CREATE TEMPORARY VIEW WEATHER USING parquet OPTIONS (path \"dbfs:/mnt/mids-w261/datasets_final_project/weather_data/weather*.parquet\")")

In [0]:
%sql
CREATE DATABASE group25;

In [0]:
%sql
DROP TABLE IF EXISTS group25.PHASE_1_PROCESSED_WEATHER;
CREATE TABLE group25.PHASE_1_PROCESSED_WEATHER AS (
  SELECT * FROM WEATHER
  WHERE CALL_SIGN LIKE ('%ORD%') OR CALL_SIGN LIKE ('%ATL%') AND REPORT_TYPE IN ('FM-15', 'FM-16') AND DATE BETWEEN '2015-01-01' AND '2015-04-01'
) 

In [0]:
 %sql
DROP TABLE IF EXISTS group25.PHASE_1_PROCESSED_AIRLINE;
CREATE TABLE group25.PHASE_1_PROCESSED_AIRLINE AS (
  SELECT *, CONCAT('K', ORIGIN, ' ') AS CALL_SIGN_A FROM AIRLINES_2015
  WHERE ORIGIN LIKE ('%ORD%') OR ORIGIN LIKE ('%ATL%') AND QUARTER=1
) 

In [0]:
%sql
-- need information on hourly basis to perform join
-- Airport codes are of 2 kinds, IATA AND ICAO - IATA is the common one (e.g. JFK, DFW)
-- ICAO is for contiguous united states, starts with "K", e.g. KJFK
-- Weather has the ICAO code, whereas airlines have the IATA code, so need to append K when doing the join

DROP TABLE IF EXISTS group25.PHASE_1_PROCESSED_WEATHER_AIRLINES;
CREATE TABLE group25.PHASE_1_PROCESSED_WEATHER_AIRLINES AS (
  SELECT * FROM group25.PHASE_1_PROCESSED_AIRLINE AS A
  LEFT JOIN (SELECT * FROM group25.PHASE_1_PROCESSED_WEATHER) AS W
  ON W.CALL_SIGN=A.CALL_SIGN_A
  AND CAST(W.DATE AS DATE)=CAST(A.FL_DATE AS DATE)
  ORDER BY W.CALL_SIGN,  A.OP_CARRIER_FL_NUM, W.DATE ASC
);

SELECT * FROM group25.PHASE_1_PROCESSED_WEATHER_AIRLINES LIMIT 10;

In [0]:
#Load joined table on to dataframe
df = sqlContext.table("group25.phase_1_processed_weather_airlines")
df.count()


In [0]:
# Downselect the columns of interest
columns = [
  'YEAR'
QUARTER:integer
MONTH:integer
DAY_OF_MONTH:integer
DAY_OF_WEEK:integer
FL_DATE:string
OP_UNIQUE_CARRIER:string
OP_CARRIER_AIRLINE_ID:integer
OP_CARRIER:string
TAIL_NUM:string
OP_CARRIER_FL_NUM:integer
ORIGIN_AIRPORT_ID:integer
ORIGIN_AIRPORT_SEQ_ID:integer
ORIGIN_CITY_MARKET_ID:integer
ORIGIN:string
ORIGIN_CITY_NAME:string
ORIGIN_STATE_ABR:string
ORIGIN_STATE_FIPS:integer
ORIGIN_STATE_NM:string
ORIGIN_WAC:integer
DEST_AIRPORT_ID:integer
DEST_AIRPORT_SEQ_ID:integer
DEST_CITY_MARKET_ID:integer
DEST:string
DEST_CITY_NAME:string
DEST_STATE_ABR:string
DEST_STATE_FIPS:integer
DEST_STATE_NM:string
DEST_WAC:integer
CRS_DEP_TIME:integer
DEP_TIME:integer
DEP_DELAY:double
DEP_DELAY_NEW:double
DEP_DEL15:double
DEP_DELAY_GROUP:integer
DEP_TIME_BLK:string
TAXI_OUT:double
WHEELS_OFF:integer
WHEELS_ON:integer
TAXI_IN:double
CRS_ARR_TIME:integer
ARR_TIME:integer
ARR_DELAY:double
ARR_DELAY_NEW:double
ARR_DEL15:double
ARR_DELAY_GROUP:integer
ARR_TIME_BLK:string
CANCELLED:double
CANCELLATION_CODE:string
DIVERTED:double
CRS_ELAPSED_TIME:double
ACTUAL_ELAPSED_TIME:double
AIR_TIME:double
FLIGHTS:double
DISTANCE:double
DISTANCE_GROUP:integer
CARRIER_DELAY:double
WEATHER_DELAY:double
NAS_DELAY:double
SECURITY_DELAY:double
LATE_AIRCRAFT_DELAY:double
FIRST_DEP_TIME:integer
TOTAL_ADD_GTIME:double
LONGEST_ADD_GTIME:double
DIV_AIRPORT_LANDINGS:integer
DIV_REACHED_DEST:double
DIV_ACTUAL_ELAPSED_TIME:double
DIV_ARR_DELAY:double
DIV_DISTANCE:double
DIV1_AIRPORT:string
DIV1_AIRPORT_ID:integer
DIV1_AIRPORT_SEQ_ID:integer
DIV1_WHEELS_ON:integer
DIV1_TOTAL_GTIME:double
DIV1_LONGEST_GTIME:double
DIV1_WHEELS_OFF:integer
DIV1_TAIL_NUM:string
DIV2_AIRPORT:string
DIV2_AIRPORT_ID:integer
DIV2_AIRPORT_SEQ_ID:integer
DIV2_WHEELS_ON:integer
DIV2_TOTAL_GTIME:double
DIV2_LONGEST_GTIME:double
DIV2_WHEELS_OFF:integer
DIV2_TAIL_NUM:string
DIV3_AIRPORT:string
DIV3_AIRPORT_ID:integer
DIV3_AIRPORT_SEQ_ID:integer
DIV3_WHEELS_ON:integer
DIV3_TOTAL_GTIME:double
DIV3_LONGEST_GTIME:double
DIV3_WHEELS_OFF:string
DIV3_TAIL_NUM:string
DIV4_AIRPORT:string
DIV4_AIRPORT_ID:string
DIV4_AIRPORT_SEQ_ID:string
DIV4_WHEELS_ON:string
DIV4_TOTAL_GTIME:string
DIV4_LONGEST_GTIME:string
DIV4_WHEELS_OFF:string
DIV4_TAIL_NUM:string
DIV5_AIRPORT:string
DIV5_AIRPORT_ID:string
DIV5_AIRPORT_SEQ_ID:string
DIV5_WHEELS_ON:string
DIV5_TOTAL_GTIME:string
DIV5_LONGEST_GTIME:string
DIV5_WHEELS_OFF:string
DIV5_TAIL_NUM:string
CALL_SIGN_A:string
STATION:string
DATE:timestamp
SOURCE:short
LATITUDE:double
LONGITUDE:double
ELEVATION:double
NAME:string
REPORT_TYPE:string
CALL_SIGN:string
QUALITY_CONTROL:string
WND:string
CIG:string
VIS:string
TMP:string
DEW:string
SLP:string
AW1:string
GA1:string
GA2:string
GA3:string
GA4:string
GE1:string
GF1:string
KA1:string
KA2:string
MA1:string
MD1:string
MW1:string
MW2:string
OC1:string
OD1:string
OD2:string
REM:string
EQD:string
AW2:string
AX4:string
GD1:string
AW5:string
GN1:string
AJ1:string
AW3:string
MK1:string
KA4:string
GG3:string
AN1:string
RH1:string
AU5:string
HL1:string
OB1:string
AT8:string
AW7:string
AZ1:string
CH1:string
RH3:string
GK1:string
IB1:string
AX1:string
CT1:string
AK1:string
CN2:string
OE1:string
MW5:string
AO1:string
KA3:string
AA3:string
CR1:string
CF2:string
KB2:string
GM1:string
AT5:string
AY2:string
MW6:string
MG1:string
AH6:string
AU2:string
GD2:string
AW4:string
MF1:string
AA1:string
AH2:string
AH3:string
OE3:string
AT6:string
AL2:string
AL3:string
AX5:string
IB2:string
AI3:string
CV3:string
WA1:string
GH1:string
KF1:string
CU2:string
CT3:string
SA1:string
AU1:string
KD2:string
AI5:string
GO1:string
GD3:string
CG3:string
AI1:string
AL1:string
AW6:string
MW4:string
AX6:string
CV1:string
ME1:string
KC2:string
CN1:string
UA1:string
GD5:string
UG2:string
AT3:string
AT4:string
GJ1:string
MV1:string
GA5:string
CT2:string
CG2:string
ED1:string
AE1:string
CO1:string
KE1:string
KB1:string
AI4:string
MW3:string
KG2:string
AA2:string
AX2:string
AY1:string
RH2:string
OE2:string
CU3:string
MH1:string
AM1:string
AU4:string
GA6:string
KG1:string
AU3:string
AT7:string
KD1:string
GL1:string
IA1:string
GG2:string
OD3:string
UG1:string
CB1:string
AI6:string
CI1:string
CV2:string
AZ2:string
AD1:string
AH1:string
WD1:string
AA4:string
KC1:string
IA2:string
CF3:string
AI2:string
AT1:string
GD4:string
AX3:string
AH4:string
KB3:string
CU1:string
CN4:string
AT2:string
CG1:string
CF1:string
GG1:string
MV2:string
CW1:string
GG4:string
AB1:string
AH5:string
CN3:string
Out[3]: Row(YEAR=2015, QUARTER=1, MON
]

In [0]:
%sql

SELECT * FROM (SELECT *, CONCAT('K', ORIGIN, ' ') AS CALL_SIGN FROM AIRLINES_2015) AS A
LEFT JOIN 
(  SELECT DISTINCT
    STATION,
    CAST(DATE AS TIMESTAMP) AS DATE,
    SOURCE,
    LATITUDE,
    LONGITUDE,
    ELEVATION,
    NAME,
    REPORT_TYPE,
    CALL_SIGN,
    QUALITY_CONTROL,
    WND,
    CIG,
    VIS,
    TMP,
    DEW,
    SLP,
    REM,
    AA1,
    AA2,
    AA3,
    AA4,
    AB1,
    AD1,
    AE1,
    AH1,
    AH2,
    AH3,
    AH4,
    AH5,
    AH6,
    AI1,
    AI2,
    AI3,
    AI4,
    AI5,
    AI6,
    AJ1,
    AK1,
    AL1,
    AL2,
    AL3 FROM WEATHER) AS W
ON W.CALL_SIGN=A.CALL_SIGN
AND CAST(W.DATE AS DATE)=CAST(A.FL_DATE AS DATE)
WHERE A.ORIGIN='ORD' AND A.QUARTER=1
ORDER BY A.ORIGIN, A.OP_CARRIER_FL_NUM, W.DATE ASC ;

In [0]:
spark.sql("CREATE TEMPORARY VIEW AIRLINES_DATA USING parquet OPTIONS (path \"dbfs:/mnt/mids-w261/datasets_final_project/parquet_airlines_data/*.parquet\")")

In [0]:
%sql
SELECT * FROM WEATHER WHERE CALL_SIGN LIKE '%KORD%' LIMIT 10
-- spark.sql("SELECT * FROM WEATHER LIMIT 10").show

In [0]:
%sql
SELECT REPORT_TYPE, COUNT(*) FROM WEATHER WHERE CALL_SIGN != '99999' GROUP BY REPORT_TYPE 

In [0]:
%sql
SELECT * FROM WEATHER WHERE REPORT_TYPE LIKE '%SAO%' AND NAME LIKE '%CHICAGO%' LIMIT 100

# Stations

In [0]:
stations = spark.read.option("header", "true").csv("dbfs:/mnt/mids-w261/DEMO8/gsod/stations.csv.gz")

In [0]:
display(stations)

In [0]:
from pyspark.sql import functions as f
stations.where(f.col('call').contains('KORD')).take(10)

In [0]:
stations.select(['name', 'call']).where(f.col('call').contains('KORD')).distinct().take(100)

In [0]:
display(stations.select('name').distinct())

In [0]:
weather.select('NAME').distinct().count()

In [0]:
display(weather.select('name').distinct())

In [0]:
spark.conf.set("spark.sql.legacy.allowCreatingManagedTableUsingNonemptyLocation","true")

In [0]:
%sql

-- DROP TABLE IF EXISTS group25.PHASE_1_PROCESSED_WEATHER;
-- CREATE TABLE group25.PHASE_1_PROCESSED_WEATHER AS (
--   SELECT * FROM WEATHER
--   WHERE CALL_SIGN LIKE ('%ORD%') OR CALL_SIGN LIKE ('%ATL%') AND REPORT_TYPE IN ('FM-15', 'FM-16') AND DATE BETWEEN '2015-01-01' AND '2015-04-01'
-- ) 


DROP TABLE IF EXISTS group25.PHASE_1_PROCESSED_AIRPORT_TRAFFIC;

CREATE TABLE group25.PHASE_1_PROCESSED_AIRPORT_TRAFFIC AS (
	SELECT * FROM group25.phase_1_processed_weather_airlines
	SORT BY ORIGIN_AIRPORT_ID, FL_DATE, CRS_DEP_TIME
)

In [0]:
REFRESH TABLE group25.PHASE_1_PROCESSED_AIRPORT_TRAFFIC

In [0]:
%sql
-- Table: outbbound_count
-- airport: ORIGIN_AIRPORT_ID
-- date: FL_DATE
-- outbound_count: COUNT(rows)

SELECT
  ORIGIN,
  ORIGIN_AIRPORT_ID,
  FL_DATE,
  CAST(DEP_TIME / 100 AS INT) AS departure_hour,
  COUNT(*) AS outbound_count
FROM group25.PHASE_1_PROCESSED_AIRPORT_TRAFFIC
GROUP BY 1, 2, 3, 4
ORDER BY 1, 2, 3, 4 ASC;

ORIGIN,ORIGIN_AIRPORT_ID,FL_DATE,departure_hour,outbound_count
ATL,10397,2015-01-01,null,48
ATL,10397,2015-01-01,5,24
ATL,10397,2015-01-01,6,144
ATL,10397,2015-01-01,7,240
ATL,10397,2015-01-01,8,792
ATL,10397,2015-01-01,9,1344
ATL,10397,2015-01-01,10,1440
ATL,10397,2015-01-01,11,744
ATL,10397,2015-01-01,12,1296
ATL,10397,2015-01-01,13,1104


In [0]:
%sql

-- SELECT
--   ORIGIN,
--   ORIGIN_AIRPORT_ID,
--   DAY_OF_WEEK,
--   CAST(DEP_TIME / 100 AS INT) AS departure_hour
--   COUNT(DISTINCT FL_DATE) AS num_of_unique_dates,
--   COUNT(*) AS outbound_count
-- FROM group25.PHASE_1_PROCESSED_AIRPORT_TRAFFIC
-- GROUP BY 1, 2, 3
-- ORDER BY 1, 2, 3 ASC;


SELECT
  ORIGIN,
  ORIGIN_AIRPORT_ID,
  DAY_OF_WEEK,
  CAST(DEP_TIME / 100 AS INT) AS departure_hour,
  COUNT(*) AS outbound_count,
  COUNT(DISTINCT FL_DATE) AS num_of_unique_dates
FROM group25.PHASE_1_PROCESSED_AIRPORT_TRAFFIC
GROUP BY 1, 2, 3, 4
ORDER BY 1, 2, 3, 4 ASC;

ORIGIN,ORIGIN_AIRPORT_ID,DAY_OF_WEEK,departure_hour,outbound_count,num_of_unique_dates
ATL,10397,1,null,10658,11
ATL,10397,1,0,749,6
ATL,10397,1,1,78,2
ATL,10397,1,5,1377,13
ATL,10397,1,6,7297,13
ATL,10397,1,7,14502,13
ATL,10397,1,8,34570,13
ATL,10397,1,9,36243,13
ATL,10397,1,10,35727,13
ATL,10397,1,11,25154,13


In [0]:
%sql
-- Table: outbbound_count
-- airport: ORIGIN_AIRPORT_ID
-- date: FL_DATE
-- outbound_count: COUNT(rows)
-- avergea_outbound_count_by_weekday_hour: AVG()
SELECT 
  ORIGIN,
  ORIGIN_AIRPORT_ID,
  DAY_OF_WEEK,
  departure_hour,
  outbound_count / num_of_unique_dates AS avg_outbound_count
FROM (
  SELECT
    ORIGIN,
    ORIGIN_AIRPORT_ID,
    DAY_OF_WEEK,
    CAST(DEP_TIME / 100 AS INT) AS departure_hour,
    COUNT(*) AS outbound_count,
    COUNT(DISTINCT FL_DATE) AS num_of_unique_dates
  FROM group25.PHASE_1_PROCESSED_AIRPORT_TRAFFIC
  GROUP BY 1, 2, 3, 4
  ORDER BY 1, 2, 3, 4 ASC
);



ORIGIN,ORIGIN_AIRPORT_ID,DAY_OF_WEEK,departure_hour,avg_outbound_count
ATL,10397,1,null,968.9090909090909
ATL,10397,1,0,124.83333333333333
ATL,10397,1,1,39.0
ATL,10397,1,5,105.92307692307692
ATL,10397,1,6,561.3076923076923
ATL,10397,1,7,1115.5384615384614
ATL,10397,1,8,2659.230769230769
ATL,10397,1,9,2787.923076923077
ATL,10397,1,10,2748.230769230769
ATL,10397,1,11,1934.923076923077


In [0]:
%sql
-- Table: inbbound_count
-- airport: ORIGIN_AIRPORT_ID
-- date: FL_DATE
-- inbound_count: COUNT(rows)

SELECT
  DEST,
  DEST_AIRPORT_ID,
  FL_DATE,
  CAST(ARR_TIME / 100 AS INT) AS arrival_hour,
  COUNT(*) AS inbound_count
FROM group25.PHASE_1_PROCESSED_AIRPORT_TRAFFIC
GROUP BY 1, 2, 3, 4
ORDER BY 1, 2, 3, 4 ASC;

DEST,DEST_AIRPORT_ID,FL_DATE,arrival_hour,inbound_count
ABE,10135,2015-01-01,15,24
ABE,10135,2015-01-01,20,26
ABE,10135,2015-01-01,22,24
ABE,10135,2015-01-02,15,32
ABE,10135,2015-01-02,21,25
ABE,10135,2015-01-02,23,32
ABE,10135,2015-01-03,15,36
ABE,10135,2015-01-03,23,36
ABE,10135,2015-01-04,18,123
ABE,10135,2015-01-04,22,51


In [0]:
%sql
-- Table: deverted_count
-- airport: ORIGIN_AIRPORT_ID
-- date: FL_DATE
-- diverted_count: COUNT(rows)

SELECT
  DEST,
  DEST_AIRPORT_ID, 
  FL_DATE,
  CAST(ARR_TIME / 100 AS INT) AS arrival_hour,
  COUNT(*) AS diverted_count
FROM group25.PHASE_1_PROCESSED_AIRPORT_TRAFFIC
WHERE DIVERTED = 1
GROUP BY 1, 2, 3, 4
ORDER BY 1, 2, 3, 4 ASC;

DEST,FL_DATE,arrival_hour,diverted_count
ABE,10135,null,68
ABE,10135,18,51
ABQ,10140,0,25
ABQ,10140,12,45
ABY,10146,null,45
ABY,10146,17,40
AEX,10185,22,23
AGS,10208,6,33
ALB,10257,13,48
ALB,10257,23,36


In [0]:
%sql

SELECT DIVERTED FROM group25.PHASE_1_PROCESSED_AIRPORT_TRAFFIC LIMIT 10;

DIVERTED
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


In [0]:
%sql 

describe group25.PHASE_1_PROCESSED_AIRPORT_TRAFFIC;

col_name,data_type,comment
YEAR,int,null
QUARTER,int,null
MONTH,int,null
DAY_OF_MONTH,int,null
DAY_OF_WEEK,int,null
FL_DATE,string,null
OP_UNIQUE_CARRIER,string,null
OP_CARRIER_AIRLINE_ID,int,null
OP_CARRIER,string,null
TAIL_NUM,string,null


In [0]:
DROP TABLE IF EXISTS group25.PHASE_1_PROCESSED_AIRPORT_TRAFFIC_Inbound_and_Outbound;

CREATE TABLE group25.PHASE_1_PROCESSED_AIRPORT_TRAFFIC AS (
	SELECT * FROM group25.phase_1_processed_weather_airlines
	SORT BY ORIGIN_AIRPORT_ID, FL_DATE, CRS_DEP_TIME
)

## Proposed Features
- Predicted variable (Y) - boolean variable, 0 if no delay, 1 if there is a delay of 15 minutes or more

## Predicted Varaibles
- Temporal
  - Day of the Week
  - Weekday vs weekend
  - Calendar Holidays (might involve brining in another dataset)
- Planning / Traffic
  - How busy is an airport might be on a particular day (how many fights are in bound)
  - Traffic needs to be normalized against itself
  - Diverted flights
  
- Delay propegation

  
- Weather *
  - 
- Airline Characteristics?
  - Large vs small airline
  - Delay history (risk score per airport/airline)
  - Centrality of the airport (page rank) - delays at a large airport may propegate to a smaller airport, at source and destination
    - Look at Origin/Destination, get a distinct list of pairs of origin/destination, generate a graph, calculate centrality measure of airports
  - Look at inbound delays, the status of these inbound flights, and the chance leading to an outbound delays 
    
## 
- Do the readings associated in git repo for flight delays
- 


## 
- Plan for the week
  - Do the readings associated in git repo for flight delays 
  - 2 separate notebooks - one for a data processing, one for modeling
  - Start with simple model with easy to grab features - Karthik
  - Start constructing more complex features
    - Weather - Jeff
      - Weather Data
      - Hour variable
    - Planning / Traffic - 
      - How busy is an airport might be on a particular day (how many flights are inbound) - how to do the joi
      - Traffic needs to be normalized against itself
      - Diverted flights
    - Centrality of the Airport (may be extra)

# __Section 1__ - Question Formulation

# __Section 2__ - Algorithm Explanation

# __Section 3__ - EDA & Challenges

# __Section 4__ - Algorithm Implementation

# __Section 5__ - Course Concepts